***
some experiments with roi geometries: geeutils.squareareaboundsroi
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils



***
__squareareaboundsroi(eepoint, metersradius, eeprojection=None)__
- aims to create square roi
- 'square' in the default-or-specified crs
- radius (half-width) specified in meters

***

In [ ]:
  
#
# ee.Geometry.Point(coords, proj) : allows creating EPSG:4326 points, e.g. ee.Geometry.Point(lng, lat)
# tapspoint = ee.Geometry.Point(5.07924, 51.21848)
#
eepoint = geeutils.tapspoint
print(geeutils.szgeometryinfo(eepoint))
#
# geeutils.squareareaboundsroi
#
rois = [geeutils.squareareaboundsroi(eepoint, metersradius) for metersradius in [100,200,300,1000]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 13)
for roi in rois:
    print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),        {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint, 10, 2), {'palette':'#000000'})
map


***
__squareareaboundsroi(eepoint, metersradius, eeprojection)__

- 'square' is relative; depends on projection
- comparing squareareaboundsroi with native projections from different collections
- note: different bands from single image can have different projections: e.g. S1 VV vs S1 angle
***

In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday
metersradius = 500

pv333image   = geeutils.someImageNear(geeutils.pv333rgbImageCollection, eedate, eepoint) # ['RED', 'NIR', 'BLUE']
s2rgbimage   = geeutils.someImageNear(geeutils.s2rgbImageCollection,    eedate, eepoint) # ['B4', 'B3', 'B2']
s1rgbimage   = geeutils.someImageNear(geeutils.s1rbgImageCollection,    eedate, eepoint) # ['VV', 'VH', 'angle']

projs = [
    [pv333image.select(0).projection(),       "pv333 RED", "#000000"], 
    [s2rgbimage.select(0).projection(),       "s2 B4",     "#0000ff"], 
    [s1rgbimage.select(0).projection(),       "s1 VV",     "#00ff00"], 
    [s1rgbimage.select('angle').projection(), "s1 angle",  "#ff0000"]]

map = geemap.Map(height='600px')
map.centerObject(eepoint, 15)
for proj in projs:
    roi = geeutils.squareareaboundsroi(eepoint, metersradius, proj[0], verbose=True)
    print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi, 0,  5), {'palette': proj[2]}, proj[1])
map.addLayer(geeutils.outlinegeometryimage(eepoint, 50, 5), {'palette':'#ff00ff'}, 'eepoint')
map
